# Create Date and Day of Week columns in the DF

In [2]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

load_dur1 = pd.read_csv('outputs/load_long_format.csv', index_col=0)
load_years = pd.read_csv('inputs/load_years.csv').dropna()
#print(load_dur1.head())
print(load_years)

           2016        2011
0    2016-01-01  2011-01-01
1    2016-01-02  2011-01-02
2    2016-01-03  2011-01-03
3    2016-01-04  2011-01-04
4    2016-01-05  2011-01-05
..          ...         ...
360  2016-12-26  2011-12-27
361  2016-12-27  2011-12-28
362  2016-12-28  2011-12-29
363  2016-12-29  2011-12-30
364  2016-12-30  2011-12-31

[365 rows x 2 columns]


In [3]:
from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = load_years['2016'].repeat(24).reset_index(drop=True)
year_2011 = load_years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = load_dur1.loc[load_dur1['R_Group'] == 'ERC'].copy()
temp2011 = load_dur1.loc[load_dur1['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

load_dur2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(load_dur2)

#convert date to a datetime type 
load_dur2['Date'] = pd.to_datetime(load_dur2['Date'])
load_dur2['DOW'] = load_dur2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
load_dur2 = pd.merge(load_dur2,weekday,on='DOW',how='left')
#print(load_dur2)

# Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours

In [4]:
case1_load = load_dur2.copy()

aggregations = {'Load':['count',sum,'mean']}
case1 = case1_load.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
#case1.to_csv('outputs/load_segments_2daytype_monthly_24hr.csv')
print()

case1_load2 = pd.merge(case1_load,case1,on=['Region','Month','Hour'],how='left')
case1_load2 = case1_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
#case1_load2.to_csv('outputs/load_8760_2daytype_monthly_24hr.csv')


     Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST      1     1        31   1042446  33627.290323
1  ERC_REST      1     2        31   1051661  33924.548387
2  ERC_REST      1     3        31   1079739  34830.290323
3  ERC_REST      1     4        31   1144492  36919.096774
4  ERC_REST      1     5        31   1239385  39980.161290
number of rows in dataset = 18144

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   

       Load       Date  DOW  Weekday  Hour_Tot  Load_Tot      Load_Avg  
2042  26989 2016-03-26    5    False        31    886708  28603.483871  
2233  27006 2016-04-03    6    False        30    870591  29019.700000  
1706  27025 2016-03-12    5    False        31    886708  28603.483871  
nu

# Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [24]:
case2_load = load_dur2.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case2_load = pd.merge(case2_load, case2_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case2 = case2_load.groupby(['Region','Season_Group','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('outputs/load_segments_2daytype_season_24hr.csv')
print()

case2_load2 = pd.merge(case2_load,case2,on=['Region','Season_Group','Hour'],how='left')
case2_load2 = case2_load2.sort_values(['Region','Load'])
print(case2_load2.head(3))
print('number of rows in dataset =',case2_load2.shape[0])
case2_load2.to_csv('outputs/load_8760_2daytype_season_24hr.csv')

     Region  Season_Group  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST             1     1        90   2928629  32540.322222
1  ERC_REST             1     2        90   2929331  32548.122222
2  ERC_REST             1     3        90   2978696  33096.622222
3  ERC_REST             1     4        90   3116896  34632.177778
4  ERC_REST             1     5        90   3346197  37179.966667
number of rows in dataset = 7560

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   

       Load       Date  DOW  Weekday  Season_Group  Hour_Tot  Load_Tot  \
2042  26989 2016-03-26    5    False             2       122   3603145   
2233  27006 2016-04-03    6    False             2       122   3587428   
1706  27025 2016-03-12    5    

# Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [6]:
case3_load = load_dur2.copy()

aggregations = {'Load':['count',sum,'mean']}
case3 = case3_load.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case3.shape[0])
#case3.to_csv('outputs/load_segments_2daytype_monthly_wkd.csv')
print()

case3_load2 = pd.merge(case3_load,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_load2 = case3_load2.sort_values(['Region','Load'])
print(case3_load2.head())
print('number of rows in dataset =',case3_load2.shape[0])
#case3_load2.to_csv('outputs/load_8760_2daytype_monthly_wkd.csv')

     Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST      1     1        31   1042446  33627.290323
1  ERC_REST      1     2        31   1051661  33924.548387
2  ERC_REST      1     3        31   1079739  34830.290323
3  ERC_REST      1     4        31   1144492  36919.096774
4  ERC_REST      1     5        31   1239385  39980.161290
number of rows in dataset = 36288

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   
1538  ERC_REST     ERC       REST  shoulder      3   65     3          1539   
1537  ERC_REST     ERC       REST  shoulder      3   65     2          1538   

       Load       Date  DOW  Weekday  Hour_Tot  Load_Tot   Load_Avg  
2042  26989 2016-03-26    5    False         8    226231  28278.87

# Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [22]:
case4_load = load_dur2.copy()
case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case4_load = pd.merge(case4_load, case4_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case4 = case4_load.groupby(['Region','Season_Group','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season_Group','Weekday','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
#case4.to_csv('outputs/load_segments_2daytype_season_wkd.csv')
print()

case4_load2 = pd.merge(case4_load,case4,on=['Region','Season_Group','Weekday','Hour'],how='left')
case4_load2 = case4_load2.sort_values(['Region','Load'])
print(case4_load2.head())
print('number of rows in dataset =',case4_load2.shape[0])
#case4_load2.to_csv('outputs/load_8760_2daytype_season_wkd.csv')

     Region  Season_Group  Weekday  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST             1    False     1        26    849384  32668.615385
1  ERC_REST             1    False     2        26    849398  32669.153846
2  ERC_REST             1    False     3        26    861202  33123.153846
3  ERC_REST             1    False     4        26    893024  34347.076923
4  ERC_REST             1    False     5        26    945126  36351.000000
number of rows in dataset = 15120

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   
1538  ERC_REST     ERC       REST  shoulder      3   65     3          1539   
1537  ERC_REST     ERC       REST  shoulder      3   65     2          1538   

       Load       Date  DOW  Weekday  Se

# 4 hour interval day types

In [27]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
print(interval_4hr)

load_dur3 = pd.merge(load_dur2,interval_4hr,on='Hour_Counter',how='left')
print(load_dur3)

      Hour_Counter  4-hr
0                1     1
1                2     1
2                3     1
3                4     1
4                5     2
...            ...   ...
8755          8756  2189
8756          8757  2190
8757          8758  2190
8758          8759  2190
8759          8760  2190

[8760 rows x 2 columns]
          Region R_Group R_Subgroup  Season  Month  Day  Hour  Hour_Counter  \
0       ERC_REST     ERC       REST  winter      1    1     1             1   
1       ERC_REST     ERC       REST  winter      1    1     2             2   
2       ERC_REST     ERC       REST  winter      1    1     3             3   
3       ERC_REST     ERC       REST  winter      1    1     4             4   
4       ERC_REST     ERC       REST  winter      1    1     5             5   
...          ...     ...        ...     ...    ...  ...   ...           ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20          8756   
551876  WEC_SDGE     WEC       SDGE  winter

# Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [28]:
case5_load = load_dur3.copy()

aggregations = {'Load':['count',sum,'mean']}
case5 = case5_load.groupby(['Region','Month','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
#case5.to_csv('outputs/load_segments_2daytype_month_4hr.csv')
print()

case5_load2 = pd.merge(case5_load,case5,on=['Region','Month','4-hr'],how='left')
case5_load2 = case5_load2.sort_values(['Region','Load'])
print(case5_load2.head())
print('number of rows in dataset =',case5_load2.shape[0])
#case5_load2.to_csv('outputs/load_8760_2daytype_month_4hr.csv')

     Region  Month  4-hr  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST      1     1         4    139677  34919.25
1  ERC_REST      1     2         4    153285  38321.25
2  ERC_REST      1     3         4    160446  40111.50
3  ERC_REST      1     4         4    155314  38828.50
4  ERC_REST      1     5         4    162472  40618.00
number of rows in dataset = 137970

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   
1538  ERC_REST     ERC       REST  shoulder      3   65     3          1539   
1537  ERC_REST     ERC       REST  shoulder      3   65     2          1538   

       Load       Date  DOW  Weekday  4-hr  Hour_Tot  Load_Tot  Load_Avg  
2042  26989 2016-03-26    5    False   511         4    108862  27215.50  
2233  270

# Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [32]:
case6_load = load_dur3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_load = pd.merge(case6_load, case6_bimonth, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case6 = case6_load.groupby(['Region','Bimonth','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
#case6.to_csv('outputs/load_segments_2daytype_bimonth_4hr.csv')
print()

case6_load2 = pd.merge(case6_load,case6,on=['Region','Bimonth','4-hr'],how='left')
case6_load2 = case6_load2.sort_values(['Region','Load'])
print(case6_load2.head())
print('number of rows in dataset =',case6_load2.shape[0])
#case6_load2.to_csv('outputs/load_8760_2daytype_bimonth_4hr.csv')

     Region  Bimonth  4-hr  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST        1     1         4    139677  34919.25
1  ERC_REST        1     2         4    153285  38321.25
2  ERC_REST        1     3         4    160446  40111.50
3  ERC_REST        1     4         4    155314  38828.50
4  ERC_REST        1     5         4    162472  40618.00
number of rows in dataset = 137970

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   
1538  ERC_REST     ERC       REST  shoulder      3   65     3          1539   
1537  ERC_REST     ERC       REST  shoulder      3   65     2          1538   

       Load       Date  DOW  Weekday  4-hr  Bimonth  Hour_Tot  Load_Tot  \
2042  26989 2016-03-26    5    False   511        2         4    108862 

# Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [33]:
case7_load = load_dur3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_load = pd.merge(case7_load, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case7 = case7_load.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season','Weekday','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case7.head())
print('number of rows in dataset =',case7.shape[0])
case7.to_csv('outputs/load_segments_2daytype_season_wkd_4hr.csv')
print()

case7_load2 = pd.merge(case7_load,case7,on=['Region','Season','Weekday','4-hr'],how='left')
case7_load2 = case7_load2.sort_values(['Region','Load'])
print(case7_load2.head())
print('number of rows in dataset =',case7_load2.shape[0])
case7_load2.to_csv('outputs/load_8760_2daytype_season_wkd_4hr.csv')

     Region    Season  Weekday  4-hr  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST  shoulder    False   385         4    108948  27237.00
1  ERC_REST  shoulder    False   386         4    118864  29716.00
2  ERC_REST  shoulder    False   387         4    131620  32905.00
3  ERC_REST  shoulder    False   388         4    135694  33923.50
4  ERC_REST  shoulder    False   389         4    142563  35640.75
number of rows in dataset = 137970

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   
1538  ERC_REST     ERC       REST  shoulder      3   65     3          1539   
1537  ERC_REST     ERC       REST  shoulder      3   65     2          1538   

       Load       Date  DOW  Weekday  4-hr  Season_Group  Hour_Tot  Load_Tot  \
2042  2